In [1]:
import pandas as pd
import numpy as np
from math import log2
from collections import Counter
import glob
import os
import re

In [2]:
# Define paths
base_path = "/Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze"
save_agent_path = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Matrices/Agent"
save_face_separated_path = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Matrices/Face_Separated"
entropy_log_faces = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/entropy_log_faces.csv"
entropy_log_combined = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/entropy_log_combined.csv"
failed_files_path = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/failed_files_log.csv"
final_matrix_faces_path = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/final_transition_matrix_faces.csv"
final_matrix_combined_path = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/final_transition_matrix_combined.csv"

# Ensure directories exist
os.makedirs(save_agent_path, exist_ok=True)
os.makedirs(save_face_separated_path, exist_ok=True)

# Define categories
categories_agent = [
    "Background", "Building", "TaskBuilding_Public", "TaskBuilding_Residential",
    "Global_Landmark", "Passive_Agent", "Active_Agent"
]

categories_face_separated = [
    "Background", "Building", "TaskBuilding_Public", "TaskBuilding_Residential",
    "Global_Landmark", "Passive_Agent_Face", "Passive_Agent", "Active_Agent_Face", "Active_Agent"
]

# Mapping dictionary
merge_face_mapping = {
    "Passive_Agent_Face": "Passive_Agent",
    "Active_Agent_Face": "Active_Agent"
}

# Initialize overall raw matrices
overall_raw_matrix_faces = pd.DataFrame(0, index=categories_face_separated, columns=categories_face_separated, dtype=float)
overall_raw_matrix_combined = pd.DataFrame(0, index=categories_agent, columns=categories_agent, dtype=float)

# Initialize entropy results and failure logs
entropy_results_faces = []
entropy_results_combined = []
failed_files = []

# Function to calculate transition entropy
def calculate_transition_entropy(matrix, stationary_distribution):
    total_entropy = 0
    category_entropies = {}
    for i, row in matrix.iterrows():
        row_entropy = sum(-p * np.log2(p) for p in row if p > 0)
        category_entropies[i] = row_entropy
        total_entropy += row_entropy * stationary_distribution.get(i, 0)
    return total_entropy, category_entropies

# Function to process and save data
def process_and_save(file, mapping_type, save_path, all_categories, mapping, entropy_results, overall_matrix):
    try:
        print(f"Processing file: {file} ({mapping_type})")

        # Load the CSV file
        One_participant = pd.read_csv(file)
        if 'Collider_CategoricalN' not in One_participant.columns:
            raise ValueError("Missing required column: 'Collider_CategoricalN'")

        # Filter for desired gaze events
        data_Reduced = One_participant[One_participant['events'] == -2].copy()
        if data_Reduced.empty:
            raise ValueError("Filtered data is empty")

        # Apply mapping if provided
        if mapping:
            data_Reduced['Collider_CategoricalN'] = data_Reduced['Collider_CategoricalN'].replace(mapping)

        # Validate categories in the data
        data_Reduced = data_Reduced[data_Reduced['Collider_CategoricalN'].isin(all_categories)]

        # Build raw transition matrix
        raw_transition_matrix = pd.DataFrame(0, index=all_categories, columns=all_categories, dtype=float)
        gaze_sequence = data_Reduced['Collider_CategoricalN'].reset_index(drop=True)
        for i in range(len(gaze_sequence) - 1):
            current_category = gaze_sequence.iloc[i]
            next_category = gaze_sequence.iloc[i + 1]
            if current_category in all_categories and next_category in all_categories:
                raw_transition_matrix.loc[current_category, next_category] += 1

        # Add raw transitions to the overall matrix
        overall_matrix += raw_transition_matrix

        # Normalize the raw transition matrix
        row_sums = raw_transition_matrix.sum(axis=1)
        normalized_matrix = raw_transition_matrix.div(row_sums.replace(0, 1), axis=0).fillna(0)

        # Calculate stationary distribution
        eigvals, eigvecs = np.linalg.eig(normalized_matrix.T)
        stationary_distribution = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
        stationary_distribution /= stationary_distribution.sum()
        stationary_distribution_dict = {all_categories[i]: stationary_distribution[i] for i in range(len(all_categories))}

        # Calculate transition entropy
        overall_transition_entropy, transition_entropy_per_category = calculate_transition_entropy(normalized_matrix, stationary_distribution_dict)

        # Calculate stationary entropy
        num_categories = len(all_categories)
        stationary_entropy_per_category = {
            category: (-stationary_distribution_dict[category] * np.log2(stationary_distribution_dict[category]))
            if stationary_distribution_dict[category] > 0 else 0
            for category in all_categories
        }

        # Normalize entropies
        normalized_overall_transition_entropy = overall_transition_entropy / np.log2(num_categories) if num_categories > 1 else 0

        # Save transition matrix
        save_matrix_path = os.path.join(save_path, f"{os.path.basename(file).replace('.csv', f'_{mapping_type}_transition_matrix.csv')}")
        raw_transition_matrix.to_csv(save_matrix_path)
        print(f"Saved matrix: {save_matrix_path}")

        # Collect entropy results
        result = {
            'Filename': os.path.basename(file),
            'Overall_Transition_Entropy': normalized_overall_transition_entropy
        }
        result.update({f'Transition_Entropy_{cat}': transition_entropy_per_category.get(cat, 0) / np.log2(num_categories) if num_categories > 1 else 0 for cat in all_categories})
        result.update({f'Stationary_Entropy_{cat}': stationary_entropy_per_category.get(cat, 0) / np.log2(num_categories) if num_categories > 1 else 0 for cat in all_categories})
        entropy_results.append(result)

    except Exception as e:
        error_message = f"Error processing file {file} ({mapping_type}): {e}"
        print(error_message)
        failed_files.append({'filename': os.path.basename(file), 'type': mapping_type, 'error': str(e)})

# Process files and accumulate matrices
files = glob.glob(base_path + "/*.csv")
if not files:
    print(f"No files found in {base_path}")

for file in files:
    process_and_save(file, "Agent", save_agent_path, categories_agent, merge_face_mapping, entropy_results_combined, overall_raw_matrix_combined)
    process_and_save(file, "Face_Separated", save_face_separated_path, categories_face_separated, None, entropy_results_faces, overall_raw_matrix_faces)

# Normalize and save final transition matrices
row_sums_faces = overall_raw_matrix_faces.sum(axis=1)
final_normalized_matrix_faces = overall_raw_matrix_faces.div(row_sums_faces.replace(0, 1), axis=0).fillna(0)
final_normalized_matrix_faces.to_csv(final_matrix_faces_path)
print(f"Final transition matrix (faces separated) saved to: {final_matrix_faces_path}")

row_sums_combined = overall_raw_matrix_combined.sum(axis=1)
final_normalized_matrix_combined = overall_raw_matrix_combined.div(row_sums_combined.replace(0, 1), axis=0).fillna(0)
final_normalized_matrix_combined.to_csv(final_matrix_combined_path)
print(f"Final transition matrix (combined) saved to: {final_matrix_combined_path}")

# Save entropy logs
if entropy_results_faces:
    entropy_df_faces = pd.DataFrame(entropy_results_faces)
    entropy_df_faces.to_csv(entropy_log_faces, index=False)
    print(f"Entropy log (faces separated) saved to: {entropy_log_faces}")

if entropy_results_combined:
    entropy_df_combined = pd.DataFrame(entropy_results_combined)
    entropy_df_combined.to_csv(entropy_log_combined, index=False)
    print(f"Entropy log (combined) saved to: {entropy_log_combined}")

# Log failed files
if failed_files:
    failed_df = pd.DataFrame(failed_files)
    failed_df.to_csv(failed_files_path, index=False)
    print(f"Failures logged to: {failed_files_path}")
else:
    print("All files processed successfully.")


Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_1.csv (Agent)
Saved matrix: /Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Matrices/Agent/1031_1_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_1.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Matrices/Face_Separated/1031_1_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_2.csv (Agent)
Saved matrix: /Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Matrices/Agent/1031_2_Agent_transition_matrix.csv
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_2.csv (Face_Separated)
Saved matrix: /Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Matrices/Face_Separated/1031_2_Face_Separated_transition_matrix.csv
Processing file: /Volumes/TwoTeras/1_Experimen


KeyboardInterrupt



In [3]:
data = pd.read_csv("/Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_1.csv")
data['date_seconds'] = pd.to_datetime(data['timeStampDataPointEnd'], unit='s')      

In [4]:
data.names.unique()

array(['pavement_O.002', 'terrain_O.001', 'Wall', 'building01_LOD1',
       'TaskBuilding_27', 'Building_100', 'Building_97',
       'maraz_cafe_collider', 'barbwire0', 'CollisionObject1',
       'Building_161', 'TaskBuilding_35', '23_Cma', 'Graffity_35',
       'Building_166', 'Fence_5', 'road_base_network.004',
       'pavement_Vb.003', 'Building_215', 'Crane_59', 'cyclone0',
       'crane_1', 'Fence_34', 'Building_162', 'Building_99', '20_Cma',
       'Complete_fence.002', 'CollisionObject2', 'Building_94',
       'Building_157', 'building01_LOD0', 'CollisionObject0',
       'terrain_E.001', 'Wall_9', 'Fence_9', 'Fence_10', 'Building_98',
       'Maraz_cafe_place.001', 'Building_148', 'TaskBuilding_5', '05_Cma',
       'pileOfClay_LOD0', 'Building_101', 'Wall_3', 'Body', 'Fence_12',
       'Hedge_9', 'Building_102', 'Lamppost_v1 (15)', 'pavement_E.008',
       'Cypress_v1_2 (1)', 'TaskBuilding_30', 'Graffity_30',
       'Lamppost_v1 (56)', 'Lamppost_v1 (11)', 'road.009', 'Building_1

In [6]:
data_Reduced = data[data['events'] == 2]
data_Reduced.reset_index(drop=True, inplace=True)

In [ ]:
data_Reduced[['date_seconds','names', 'Collider_CategoricalN']].head(60)

In [ ]:
data_Reduced['Collider_CategoricalN'].unique()

In [10]:
# Filter the DataFrame for rows where Collider_CategoricalN equals "agent face"
agent_face_index = data_Reduced[data_Reduced['Collider_CategoricalN'] == 'Active_Agent_Face'].index

# Check if there are any matches
if not agent_face_index.empty:
    # Take the first occurrence of "agent face"
    first_occurrence_index = agent_face_index[0]
    
    # Get the next 50 rows after the "agent face" row
    result = data_Reduced.iloc[first_occurrence_index -5:first_occurrence_index + 25]
    
    # Display the result
    display(result[['date_seconds','names', 'Collider_CategoricalN']])
else:
    print("No rows with Collider_CategoricalN equal to 'agent face' found.")

,date_seconds,names,Collider_CategoricalN
116,2022-11-14 14:28:27.774491904,Graffity_30,TaskBuilding_Residential
117,2022-11-14 14:28:27.930236416,20_Cma,Active_Agent
118,2022-11-14 14:28:28.019020032,Complete_fence.002,Background
119,2022-11-14 14:28:28.274459648,20_Cma,Active_Agent
120,2022-11-14 14:28:28.474844160,20_Cma,Active_Agent
121,2022-11-14 14:28:28.740700672,20_Cma,Active_Agent_Face
122,2022-11-14 14:28:29.207437568,Complete_fence.002,Background
123,2022-11-14 14:28:29.396413440,Complete_fence.002,Background
124,2022-11-14 14:28:29.507517696,Complete_fence.002,Background
125,2022-11-14 14:28:29.718317568,Complete_fence.002,Background


In [13]:
# Step 1: Rename the column Collider_CategoricalN to Face_separated
result.rename(columns={'Collider_CategoricalN': 'Face_separated'}, inplace=True)

# Step 2: Create a new column `combined` with "face" replaced by an empty string
result['Combined'] = result['Face_separated'].str.replace('_Face', '', regex=False)



/var/folders/3p/lxg47mnx0ks_st66vkzkx1640000gn/T/ipykernel_6094/1943744912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.rename(columns={'Collider_CategoricalN': 'Face_separated'}, inplace=True)
/var/folders/3p/lxg47mnx0ks_st66vkzkx1640000gn/T/ipykernel_6094/1943744912.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Combined'] = result['Face_separated'].str.replace('_Face', '', regex=False)


In [ ]:
result[['date_seconds','names', 'Face_separated']]

In [14]:
result[['date_seconds','names', 'Face_separated', 'Combined']]

,date_seconds,names,Face_separated,Combined
116,2022-11-14 14:28:27.774491904,Graffity_30,TaskBuilding_Residential,TaskBuilding_Residential
117,2022-11-14 14:28:27.930236416,20_Cma,Active_Agent,Active_Agent
118,2022-11-14 14:28:28.019020032,Complete_fence.002,Background,Background
119,2022-11-14 14:28:28.274459648,20_Cma,Active_Agent,Active_Agent
120,2022-11-14 14:28:28.474844160,20_Cma,Active_Agent,Active_Agent
121,2022-11-14 14:28:28.740700672,20_Cma,Active_Agent_Face,Active_Agent
122,2022-11-14 14:28:29.207437568,Complete_fence.002,Background,Background
123,2022-11-14 14:28:29.396413440,Complete_fence.002,Background,Background
124,2022-11-14 14:28:29.507517696,Complete_fence.002,Background,Background
125,2022-11-14 14:28:29.718317568,Complete_fence.002,Background,Background


In [ ]:
entropy_df.head()

In [ ]:
entropy_df.Overall_Transition_Entropy.describe()

In [ ]:
trials_df.head()